<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Summary" data-toc-modified-id="Summary-1">Summary</a></span></li><li><span><a href="#On-dataframes" data-toc-modified-id="On-dataframes-2">On dataframes</a></span></li><li><span><a href="#Data-Source-Info" data-toc-modified-id="Data-Source-Info-3">Data Source Info</a></span></li><li><span><a href="#Creating-a-data-frame" data-toc-modified-id="Creating-a-data-frame-4">Creating a data frame</a></span></li><li><span><a href="#Reading-a-data-frame" data-toc-modified-id="Reading-a-data-frame-5">Reading a data frame</a></span><ul class="toc-item"><li><span><a href="#Data-frame-structure" data-toc-modified-id="Data-frame-structure-5.1">Data frame structure</a></span></li></ul></li><li><span><a href="#Exercises" data-toc-modified-id="Exercises-6">Exercises</a></span><ul class="toc-item"><li><span><a href="#4.1" data-toc-modified-id="4.1-6.1">4.1</a></span></li><li><span><a href="#4.2" data-toc-modified-id="4.2-6.2">4.2</a></span></li></ul></li><li><span><a href="#Exploring-the-shape-of-our-data-universe" data-toc-modified-id="Exploring-the-shape-of-our-data-universe-7">Exploring the shape of our data universe</a></span><ul class="toc-item"><li><span><a href="#About-Star-Schema" data-toc-modified-id="About-Star-Schema-7.1">About Star Schema</a></span></li><li><span><a href="#select-ing-what-we-want-to-see" data-toc-modified-id="select-ing-what-we-want-to-see-7.2"><code>select</code>-ing what we want to see</a></span></li><li><span><a href="#drop-ing-columns-we-don't-need" data-toc-modified-id="drop-ing-columns-we-don't-need-7.3"><code>drop</code>-ing columns we don't need</a></span></li></ul></li><li><span><a href="#Exercises" data-toc-modified-id="Exercises-8">Exercises</a></span><ul class="toc-item"><li><span><a href="#4.3" data-toc-modified-id="4.3-8.1">4.3</a></span></li></ul></li><li><span><a href="#Creating-new-columns-with-withColumn" data-toc-modified-id="Creating-new-columns-with-withColumn-9">Creating new columns with <code>withColumn</code></a></span><ul class="toc-item"><li><span><a href="#1.-Check-the-data-type-of-'Duration'-column" data-toc-modified-id="1.-Check-the-data-type-of-'Duration'-column-9.1">1. Check the data type of 'Duration' column</a></span></li><li><span><a href="#2.-Extract-time-features-from-Duration-column-only-show-distinct" data-toc-modified-id="2.-Extract-time-features-from-Duration-column-only-show-distinct-9.2">2. Extract time features from Duration column only show distinct</a></span></li><li><span><a href="#3.-Use-withColumn()-to-add-'duration_seconds'-to-original-data-frame" data-toc-modified-id="3.-Use-withColumn()-to-add-'duration_seconds'-to-original-data-frame-9.3">3. Use <code>withColumn()</code> to add 'duration_seconds' to original data frame</a></span></li></ul></li><li><span><a href="#Batch-renaming-with-toDF()" data-toc-modified-id="Batch-renaming-with-toDF()-10">Batch renaming with <code>toDF()</code></a></span></li><li><span><a href="#Sorting-column-order-with-sort" data-toc-modified-id="Sorting-column-order-with-sort-11">Sorting column order with <code>sort</code></a></span></li><li><span><a href="#Getting-high-level-summary-of-your-dataframe-with-describe-and-summary" data-toc-modified-id="Getting-high-level-summary-of-your-dataframe-with-describe-and-summary-12">Getting high level summary of your dataframe with <code>describe</code> and <code>summary</code></a></span></li></ul></div>

# Analyzing tabular data with pyspark.sql

## Summary

- PySpark uses the `SparkReader` object to read any kind of data directly in a data frame. The specialized `CSV SparkReader` is used to ingest comma-separated value (CSV) files. Just like when reading text, the only mandatory parameter is the source location.
- The CSV format is very versatile, so PySpark provides many optional parameters to account for this flexibility. The most important ones are the `field` delimiter, the `record` delimiter, and the `quotation` character. All of those parameters have sensible defaults.
- PySpark can infer the Schema of a CSV file by setting the `inferSchema` optional parameter to True. PySpark accomplishes this by reading the data twice: once for setting the appropriate types for each columns, and another time to ingest the data in the inferred format.
- Tabular data is represented into a data frame in a series of Columns, each having a name and a type. Since the data frame is a column-major data structure, the concept of row is less relevant.
- You can use Python code to explore the data efficiently, using the column list as any Python list to expose the elements of the data frame of interest.
- The most common operations on a data frame are the selection, deletion, and creation or columns. In PySpark, the methods used are `select()`, `drop()` and `withColumn()`, respectively.
- select can be used for column re-ordering by passing a re-ordered list of columns.
- You can rename columns one by one with the `withColumnRenamed()` method, or all at once by using the `toDF()` method.
- You can display a summary of the columns with the `describe()` or `summary()` method. `describe()` has a fixed set of metrics, while `summary()` will take functions as parameters and apply them to all columns.

## On dataframes
> PySpark operates either on the whole __data frame__ objects (via methods such as `select()` and `groupby()`) or on __Column__ objects (for instance when using a function like `split()`). 
>
> - The data frame is __column-major__, so its API focuses on manipulating the columns to transform the data. 
> - Hence with data transformations, think about what operations to do and which columns will be impacted.

- RDDs on the other hand are _row-major_.  Hence you're thinking about items with attributes in which you apply functions.

In [1]:
# setup
import os
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

## Data Source Info

> For this exercise, we’ll use some open data from the Government of Canada, more specifically the CRTC (Canadian Radio-television and Telecommunications Commission). Every broadcaster is mandated to provide a complete log of the programs, commercials and all, showcased to the Canadian public. 
>
> This gives us a lot of potential questions to answer, but we’ll select one specific one: __what are the channels with the most and least proportion of commercials?__

## Creating a data frame

`spark.createDataFrame`
- 1st param: data (list of lists, pandas dataframe, RDD)
- 2nd param: schema (ie. think column headers in SQL)
- Master node knows the structure of the dataframe, but actual data is on worker nodes (ie. cluster memory)

In [4]:
# Example creating a data frame with toy data
my_grocery_list = [
    ["Banana", 2, 1.74],
    ["Apple", 4, 2.04],
    ["Carrot", 1, 1.09],
    ["Cake", 1, 10.99],
]

df_grocery_list = spark.createDataFrame(my_grocery_list, ["Item", "Quantity", "Price"])

df_grocery_list.printSchema()

root
 |-- Item: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Price: double (nullable = true)



## Reading a data frame

### Data frame structure

Composed of _row delimiter_ (e.g. newline `\n`) and _column delimiter_ (e.g. tabs `\t` for TSVs)

In [48]:
DIRECTORY = "./data/Ch04"
logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
    sep="|",  # default is ","
    quote='"',  # default is double quote.
    header=True,  # set first row as column names
    inferSchema=True,  # infer schema from column names default False
)

In [49]:
logs.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: string (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: string (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nulla

--- 

##  Exercises

### 4.1
Take the following file, called sample.csv, and read it into a dataframe.

```
Item,Quantity,Price
$Banana, organic$,1,0.99
Pear,7,1.24
$Cake, chocolate$,1,14.50
```


In [17]:
sample = spark.read.csv(
    os.path.join(DIRECTORY, "ch4_exercise.csv"),
    sep=",",
    header=True,
    quote="$",
    inferSchema=True,
)

sample.show()

+---------------+--------+-----+
|           Item|Quantity|Price|
+---------------+--------+-----+
|Banana, organic|       1| 0.99|
|           Pear|       7| 1.24|
|Cake, chocolate|       1| 14.5|
+---------------+--------+-----+



### 4.2

Re-read the data in a `logs_raw` data frame, taking inspiration from the code in listing 4.3, this time without passing any optional parameters. Print the first 5 rows of data, as well as the schema. What are the differences in terms of data and schema between logs and logs_raw?

In [46]:
DIRECTORY = "./data/Ch04"
raw_logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
)
raw_logs.show(5, False)  # False = show entire contents
raw_logs.printSchema()

# Result shows entire row concatenated into one column (_c0). Not what we want.

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0                                                                                                                                                                                                                                                                                                                                                                                                                                        |
+-------------------------------------------------------------------------------------------------------------------------------------------

---

## Exploring the shape of our data universe

### About Star Schema


Wiki:
> In computing, the __star schema__ is the simplest style of data mart schema and is the approach most widely used to develop data warehouses and dimensional data marts. The star schema consists of one or more fact tables referencing any number of dimension tables.

Star schemas are common in the relational database world because of __normalization__, a process used to avoid duplicating pieces of data and improve data integrity.

Spark uses __denormalized__ tables (ie __fat__ tables). Why? Mainly because it is easier to run analyses on a single table.  
  - If you do need to analyze complex star schema, best bet is to work with a database manger to get a denormalized table.
  
### `select`-ing what we want to see

Four ways to `select` colums in PySpark, all equivalent in term of results

In [50]:
# Using the string to column conversion
logs.select("BroadCastLogID", "LogServiceID", "LogDate")
logs.select(
    *["BroadCastLogID", "LogServiceID", "LogDate"]
)  # Unpack list with star prefix

# Passing the column object explicitly
logs.select(F.col("BroadCastLogID"), F.col("LogServiceID"), F.col("LogDate"))
logs.select(
    *[F.col("BroadCastLogID"), F.col("LogServiceID"), F.col("LogDate")]
)  # Unpack list with star prefix

DataFrame[BroadCastLogID: int, LogServiceID: int, LogDate: string]

Because of the width of our data frame, we could split our columns into manageable sets of three to keep the output tidy on the screen. This gives a high-level view of what the data frame contains. 

In [51]:
# Splitting columns in groups of three using numpy
display("Columns in groups of three")
column_split = np.array_split(np.array(logs.columns), len(logs.columns) // 3)
display(column_split)

# Show columns in groups of three
display("Table display in column groups of three")
for x in column_split:
    logs.select(*x).show(5, False)

'Columns in groups of three'

[array(['BroadcastLogID', 'LogServiceID', 'LogDate'], dtype='<U22'),
 array(['SequenceNO', 'AudienceTargetAgeID', 'AudienceTargetEthnicID'],
       dtype='<U22'),
 array(['CategoryID', 'ClosedCaptionID', 'CountryOfOriginID'], dtype='<U22'),
 array(['DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID'],
       dtype='<U22'),
 array(['ProgramClassID', 'FilmClassificationID', 'ExhibitionID'],
       dtype='<U22'),
 array(['Duration', 'EndTime', 'LogEntryDate'], dtype='<U22'),
 array(['ProductionNO', 'ProgramTitle', 'StartTime'], dtype='<U22'),
 array(['Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID'],
       dtype='<U22'),
 array(['BroadcastOriginPointID', 'CompositionID', 'Producer1'],
       dtype='<U22'),
 array(['Producer2', 'Language1', 'Language2'], dtype='<U22')]

'Table display in column groups of three'

+--------------+------------+----------+
|BroadcastLogID|LogServiceID|LogDate   |
+--------------+------------+----------+
|1196192316    |3157        |2018-08-01|
|1196192317    |3157        |2018-08-01|
|1196192318    |3157        |2018-08-01|
|1196192319    |3157        |2018-08-01|
|1196192320    |3157        |2018-08-01|
+--------------+------------+----------+
only showing top 5 rows

+----------+-------------------+----------------------+
|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|
+----------+-------------------+----------------------+
|1         |4                  |null                  |
|2         |null               |null                  |
|3         |null               |null                  |
|4         |null               |null                  |
|5         |null               |null                  |
+----------+-------------------+----------------------+
only showing top 5 rows

+----------+---------------+-----------------+
|CategoryID|ClosedCaptionID|Co

### `drop`-ing columns we don't need

Remove `BroadCastLogID` (primary key not needed in single table) and `SequenceNo`.  `drop()` returns a new data frame.

> __Warning with `drop`__: Unlike `select()`, where selecting a column that doesn’t exist will return a runtime error, dropping a non-existent column is a no-op. PySpark will __just ignore the columns it doesn’t find__. Careful with the spelling of your column names!

In [53]:
logs = logs.drop("BroadCastLogID", "SequenceNo")

assert all(col not in logs.columns for col in ["BroadCastLogID", "SequenceNo"])

Alternate method of above just using `select` using list comprehension.

In [52]:
logs = logs.select(
    *[col for col in logs.columns if col not in ["BroadCastLogID", "SequenceNo"]]
)

assert all(col not in logs.columns for col in ["BroadCastLogID", "SequenceNo"])

---

## Exercises

### 4.3

Create a new data frame logs_clean that contains only the columns that do not end with ID

In [74]:
print([col for col in logs.columns if col[-2:] != "ID"])

['LogDate', 'SequenceNO', 'Duration', 'EndTime', 'LogEntryDate', 'ProductionNO', 'ProgramTitle', 'StartTime', 'Subtitle', 'Producer1', 'Producer2', 'Language1', 'Language2']


In [98]:
# Load original CSV again
DIRECTORY = "./data/Ch04"
logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
    sep="|",  # default is ","
    quote='"',  # default is double quote.
    header=True,  # set first row as column names
    inferSchema=True,  # infer schema from column names default False
)

# Filter to columns that don't end with "ID"
logs_no_id = logs.select(*[col for col in logs.columns if col[-2:].lower() != "id"])
print("Filtered results (not end with 'ID')")
logs_no_id.printSchema()

assert all("id" not in col[-2:] for col in logs_no_id.columns)

Filtered results (not end with 'ID')
root
 |-- LogDate: string (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: string (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- Producer1: string (nullable = true)
 |-- Producer2: string (nullable = true)
 |-- Language1: integer (nullable = true)
 |-- Language2: integer (nullable = true)



## Creating new columns with `withColumn`

### 1. Check the data type of 'Duration' column

In [103]:
logs.select(F.col("Duration")).show(5)

print(
    "dtype of 'Duration' column is 'string'. Best to convert to timestamp:\n",
    logs.select(F.col("Duration")).dtypes,
)

+----------------+
|        Duration|
+----------------+
|02:00:00.0000000|
|00:00:30.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
+----------------+
only showing top 5 rows

dtype of 'Duration' column is 'string'. Best to convert to timestamp:
 [('Duration', 'string')]


### 2. Extract time features from Duration column only show distinct

In [116]:
logs.select(
    F.col("Duration"),
    F.col("Duration").substr(1, 2).cast("int").alias("hours"),
    F.col("Duration").substr(4, 2).cast("int").alias("minutes"),
    F.col("Duration").substr(7, 2).cast("int").alias("seconds"),
    # Add final column converting duration into total seconds
    (
        F.col("Duration").substr(1, 2).cast("int") * 60 * 60
        + F.col("Duration").substr(4, 2).cast("int") * 60
        + F.col("Duration").substr(7, 2).cast("int")
    ).alias("duration_seconds"),
).distinct().show(
    5
)  # only show distinct entries

+----------------+-----+-------+-------+----------------+
|        Duration|hours|minutes|seconds|duration_seconds|
+----------------+-----+-------+-------+----------------+
|00:00:19.0000000|    0|      0|     19|              19|
|00:07:09.0000000|    0|      7|      9|             429|
|00:53:26.0000000|    0|     53|     26|            3206|
|00:30:43.0000000|    0|     30|     43|            1843|
|00:02:41.0000000|    0|      2|     41|             161|
+----------------+-----+-------+-------+----------------+
only showing top 5 rows



### 3. Use `withColumn()` to add 'duration_seconds' to original data frame

In [122]:
logs = logs.withColumn(
    "duration_seconds",
    F.col("Duration").substr(1, 2).cast("int") * 60 * 60
    + F.col("Duration").substr(4, 2).cast("int") * 60
    + F.col("Duration").substr(7, 2).cast("int"),
)

assert "duration_seconds" in logs.columns

> __Warning__: If you’re creating a column withColumn() and give it a name that already exists in your data frame, PySpark will happily overwrite the column.

## Batch renaming with `toDF()`

In [124]:
logs.toDF(*[x.lower() for x in logs.columns]).printSchema()

root
 |-- broadcastlogid: integer (nullable = true)
 |-- logserviceid: integer (nullable = true)
 |-- logdate: string (nullable = true)
 |-- sequenceno: integer (nullable = true)
 |-- audiencetargetageid: integer (nullable = true)
 |-- audiencetargetethnicid: integer (nullable = true)
 |-- categoryid: integer (nullable = true)
 |-- closedcaptionid: integer (nullable = true)
 |-- countryoforiginid: integer (nullable = true)
 |-- dubdramacreditid: integer (nullable = true)
 |-- ethnicprogramid: integer (nullable = true)
 |-- productionsourceid: integer (nullable = true)
 |-- programclassid: integer (nullable = true)
 |-- filmclassificationid: integer (nullable = true)
 |-- exhibitionid: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- endtime: string (nullable = true)
 |-- logentrydate: string (nullable = true)
 |-- productionno: string (nullable = true)
 |-- programtitle: string (nullable = true)
 |-- starttime: string (nullable = true)
 |-- subtitle: string (nulla

## Sorting column order with `sort`

In [125]:
logs.select(sorted(logs.columns)).printSchema()

root
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- BroadcastLogID: integer (nullable = true)
 |-- BroadcastOriginPointID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CompositionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- Language1: integer (nullable = true)
 |-- Language2: integer (nullable = true)
 |-- LogDate: string (nullable = true)
 |-- LogEntryDate: string (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- Producer1: string (nullable = true)
 |-- Producer2: str

## Getting high level summary of your dataframe with `describe` and `summary`
- `describe` only works for numerical and string columns

In [127]:
# Show stats for the first three columns
for i in logs.columns[:3]:
    logs.describe(i).show()

+-------+--------------------+
|summary|      BroadcastLogID|
+-------+--------------------+
|  count|              238945|
|   mean|1.2168651122760174E9|
| stddev| 1.496913424143109E7|
|    min|          1195788151|
|    max|          1249431576|
+-------+--------------------+

+-------+------------------+
|summary|      LogServiceID|
+-------+------------------+
|  count|            238945|
|   mean| 3450.890284375065|
| stddev|199.50673962555592|
|    min|              3157|
|    max|              3925|
+-------+------------------+

+-------+----------+
|summary|   LogDate|
+-------+----------+
|  count|    238945|
|   mean|      null|
| stddev|      null|
|    min|2018-08-01|
|    max|2018-08-01|
+-------+----------+



- `summary` shows extra stats like 25-50% and 75% percentiles

In [135]:
# Show stats for the first three columns
for i in logs.columns[:3]:
    logs.select(i).summary().show()

+-------+--------------------+
|summary|      BroadcastLogID|
+-------+--------------------+
|  count|              238945|
|   mean|1.2168651122760174E9|
| stddev| 1.496913424143109E7|
|    min|          1195788151|
|    25%|          1249431576|
|    50%|          1213242718|
|    75%|          1226220081|
|    max|          1249431576|
+-------+--------------------+

+-------+------------------+
|summary|      LogServiceID|
+-------+------------------+
|  count|            238945|
|   mean| 3450.890284375065|
| stddev|199.50673962555592|
|    min|              3157|
|    25%|              3287|
|    50%|              3379|
|    75%|              3627|
|    max|              3925|
+-------+------------------+

+-------+----------+
|summary|   LogDate|
+-------+----------+
|  count|    238945|
|   mean|      null|
| stddev|      null|
|    min|2018-08-01|
|    25%|      null|
|    50%|      null|
|    75%|      null|
|    max|2018-08-01|
+-------+----------+

